In [2]:

import numpy as np
import torch
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image
import requests
import matplotlib.pyplot as plt

def load_model_and_image():
    """Load a pre-trained ViT model and a sample image."""
    # Load the model
    processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k')
    model = ViTForImageClassification.from_pretrained('farleyknight/mnist-digit-classification-2022-09-04')

    # Load a sample image
    url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
    raw_image = Image.open(requests.get(url, stream=True).raw)
    image = torch.tensor(np.array(raw_image)).permute(2, 0, 1)

    return processor, model, image
processor, model, image = load_model_and_image()


In [17]:
import torchvision
from torch.utils.data import TensorDataset, random_split
from random import shuffle
import numpy as np
import torch
from matplotlib import pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from torch import nn
import numpy as np
def plot_square(wcopy, **kwargs):
    ns = np.ceil(np.sqrt(wcopy.shape[0])).astype(int)
    s = np.zeros(ns**2)
    s[:wcopy.size] = wcopy
    s = s.reshape(ns,ns)
    plt.imshow(s, **kwargs)
# mms = MinMaxScaler()
def get_data(target_digit1=9, target_digit2=1, convert_y=False):
    pca = PCA(n_components=4)

    mnist = fetch_openml('mnist_784')
    X_orig, y_orig = mnist["data"], mnist["target"]

    X_orig = np.clip(np.array(X_orig).astype(np.float32) / 255.0, 0.0, 1.0)
    y_orig = np.array(y_orig).astype(np.uint8)

    target_digit1_xdata = X_orig[y_orig == target_digit1]
    target_digit2_xdata = X_orig[y_orig == target_digit2]#[:300]
    target_digit1_ydata = y_orig[y_orig == target_digit1]
    target_digit2_ydata = y_orig[y_orig == target_digit2]#[:300]
    X = np.concatenate((target_digit1_xdata, target_digit2_xdata), axis=0)
    y = np.concatenate((target_digit1_ydata, target_digit2_ydata), axis=0)

    if convert_y:
        y = np.where(y == target_digit1, -1, 1)
    return X, y

X, y = get_data(3, 8)

device = 'cpu'
device = 'cpu'
# Convert to float for image data and long for classification labels
X = torch.tensor(X, dtype=torch.float32).to(device)
y = torch.tensor(y, dtype=torch.long).to(device)
y

tensor([3, 3, 3,  ..., 8, 8, 8])

In [18]:
# --- 2. Reshape, Expand Channels, and Upscale ---

# Reshape from (N, 784) to (N, 28, 28)
X = X.reshape(-1, 28, 28)

# Add a channel dimension: (N, 28, 28) -> (N, 1, 28, 28)
X = X.unsqueeze(1)

# Repeat the channel dimension 3 times: (N, 1, 28, 28) -> (N, 3, 28, 28)
X = X.repeat(1, 3, 1, 1)

# Define and apply the upscale transformation
resize_transform = torchvision.transforms.Resize((224, 224), antialias=True)
X = resize_transform(X)

print(f"Final shape of X: {X.shape}") # Should be (N, 3, 224, 224)

# --- 3. Create Splits using PyTorch Utilities ---

# Combine images and labels into a single dataset
full_dataset = TensorDataset(X, y)

# Define the sizes for your splits (60% train, 20% validation, 20% test)
dataset_size = len(full_dataset)
train_size = int(0.6 * dataset_size)
val_size = int(0.2 * dataset_size)
test_size = dataset_size - train_size - val_size # Ensure all data is used

# Perform the split
# Use a generator for reproducible splits
generator = torch.Generator().manual_seed(42)
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size], generator=generator
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

# You are now ready to use these datasets with PyTorch DataLoaders
# For example:
# from torch.utils.data import DataLoader
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

Final shape of X: torch.Size([13966, 3, 224, 224])
Train dataset size: 8379
Validation dataset size: 2793
Test dataset size: 2794


In [19]:
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=8379)

for X, y in train_loader:
    print(X.size(), y.size())
    break

torch.Size([8379, 3, 224, 224]) torch.Size([8379])


In [21]:
X, y = train_dataset[0]
logits = model(X.unsqueeze(0)).logits
print(y.item(), logits.argmax(-1).item())

3 3


ValueError: Input image size (28*28) doesn't match model (224*224).